In [2]:
import torch
import torch.nn as nn

In [7]:
# input1 = torch.randn(100, 128)
# input2 = torch.randn(100, 128)
input1 = torch.tensor([0.0,1.0, 0.0])
input2 = torch.tensor([0.0,-1.0, 1.0])
cos = nn.CosineSimilarity(dim=0, eps=1e-6)
output = cos(input1, input2)

In [8]:
output

tensor(-0.7071)